<a href="https://colab.research.google.com/github/alexlopespereira/machine_learning/blob/main/Notebooks/Aula1/Aula1_RendaCensoUSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classificação do Adult Census Income com Rede Neural (MLPClassifier)

Este notebook demonstra como utilizar uma rede neural (MLPClassifier do scikit-learn) para resolver o problema de classificação do dataset Adult Census Income. O fluxo abrange:
- Carregamento e pré-processamento dos dados
- Divisão dos dados em treino e teste
- Transformação dos dados (padronização e One-Hot Encoding)
- Treinamento do modelo
- Avaliação do modelo

In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score

## Carregamento e Pré-processamento do Dataset

Neste passo, carregamos o dataset 'Adult' do OpenML, substituímos os valores "?" por NaN e removemos as linhas com dados faltantes. Em seguida, separamos as features da variável target e mapeamos os rótulos para valores numéricos.

In [3]:
# Carrega o dataset 'adult' do OpenML
adult = fetch_openml('adult', version=2, as_frame=True)
df = adult.frame

# Substitui valores "?" por NaN e remove linhas com dados faltantes
df = df.replace('?', np.nan).dropna()

# Separa features (variáveis preditoras) e target (rótulo)
X = df.drop(columns='class')
y = df['class'].map({'<=50K': 0, '>50K': 1}) # O que esse código faz?

## Divisão dos Dados em Treino e Teste

Utilizamos o `train_test_split` para dividir os dados em 80% para treino e 20% para teste, mantendo a proporção das classes com o parâmetro `stratify`.

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

## Pré-processamento dos Dados: Padronização e One-Hot Encoding

Primeiro, padronizamos as features numéricas utilizando o `StandardScaler`. Em seguida, aplicamos One-Hot Encoding às features categóricas usando o `OneHotEncoder`.

Por fim, concatenamos as features transformadas em um único array para o treinamento do modelo.

In [7]:
# Define as colunas numéricas e categóricas
numeric_features = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
categorical_features = [col for col in X.columns if col not in numeric_features]

# Padroniza as colunas numéricas
scaler = StandardScaler()
X_train_num = scaler.fit_transform(X_train[numeric_features])
X_test_num = scaler.transform(X_test[numeric_features])

# Aplica One-Hot Encoding nas colunas categóricas
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
X_train_cat = encoder.fit_transform(X_train[categorical_features])
X_test_cat = encoder.transform(X_test[categorical_features])

# Concatena as features numéricas e categóricas
X_train_transformed = np.concatenate([X_train_num, X_train_cat], axis=1)
X_test_transformed = np.concatenate([X_test_num, X_test_cat], axis=1)

## Definição e Treinamento do Modelo de Rede Neural

Utilizamos o `MLPClassifier` do scikit-learn para definir uma rede neural com duas camadas ocultas (64 e 32 neurônios, respectivamente). Configuramos a função de ativação `relu`, o solver `adam` e ativamos o early stopping para evitar overfitting.

In [8]:
model = MLPClassifier(hidden_layer_sizes=(64, 32),
                      activation='relu',
                      solver='adam',
                      max_iter=200,
                      random_state=42)

# Treina o modelo
model.fit(X_train_transformed, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(64, 32), random_state=42)

## Avaliação do Modelo

Realizamos a avaliação do modelo utilizando o conjunto de teste. Calculamos a acurácia e geramos um relatório de classificação com métricas detalhadas (precisão, recall, f1-score) para cada classe.

In [9]:
# Realiza as previsões no conjunto de teste
y_pred = model.predict(X_test_transformed) # O que é o argumento desta função? O que é o resultado? Qual o tipo e o shape da entrada e da saída?

# Calcula a acurácia do modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia no conjunto de teste: {accuracy:.4f}")

# Gera o relatório de classificação
print(classification_report(y_test, y_pred))

Acurácia no conjunto de teste: 0.8279
              precision    recall  f1-score   support

           0       0.87      0.91      0.89      6803
           1       0.68      0.59      0.63      2242

    accuracy                           0.83      9045
   macro avg       0.77      0.75      0.76      9045
weighted avg       0.82      0.83      0.82      9045



## Exercício 1: Faça uma nova iteração sobre este modelo alterando o argumento learning_rate_init do MLPClassifier e o rescalador para o MinMaxScaler do sklearn

## Exercício 2: Utilize alguma técnica de Fill NA para tentar melhorar a acuráciado modelo. Repita todas as etapas e hiperparametros que melhor desempenharam nos seus experimentos anteriores.